In [ ]:
import numpy as np 
import pandas as pd 



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import InputLayer, Dense, BatchNormalization, LeakyReLU, Dropout
from keras.optimizers import Adam
from keras.losses import MeanSquaredError

In [ ]:
df = pd.read_csv("../input/train-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

df1 = pd.read_csv("../input/starter-stack/train_pred_1.csv")
df2 = pd.read_csv("../input/starter-stack/train_pred_2.csv")

df_test1 = pd.read_csv("../input/starter-stack/test_pred_1.csv")
df_test2 = pd.read_csv("../input/starter-stack/test_pred_2.csv")

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")

df.head()

In [ ]:
!pip install optuna

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import optuna

In [ ]:
useful_features = ["pred_1", "pred_2"]
df_test = df_test[useful_features]

In [ ]:
# def run(trial):
#     rmses = []
#     for fold in range(5):
#         learning_rate = trial.suggest_float("learning_rate", 1e-4, 0.25, log=True)
#         reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
#         reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
#         subsample = trial.suggest_float("subsample", 0.1, 1.0)
#         colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
#         max_depth = trial.suggest_int("max_depth", 1, 7)

#         xtrain = df[df.kfold != fold].reset_index(drop=True)
#         xvalid = df[df.kfold == fold].reset_index(drop=True)

#         ytrain = xtrain.target
#         yvalid = xvalid.target

#         xtrain = xtrain[useful_features]
#         xvalid = xvalid[useful_features]

#         model = XGBRegressor(
#             random_state=42,
#             tree_method="gpu_hist",
#             gpu_id=1,
#             predictor="gpu_predictor",
#             learning_rate=learning_rate,
#             reg_lambda=reg_lambda,
#             reg_alpha=reg_alpha,
#             subsample=subsample,
#             colsample_bytree=colsample_bytree,
#             max_depth=max_depth,
#         )
#         model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
#         preds_valid = model.predict(xvalid)
#         rmse = mean_squared_error(yvalid, preds_valid, squared=False)
#         rmses.append(rmse)
#     return np.mean(rmses)

# Stack 1

In [ ]:
# study = optuna.create_study(direction="minimize")
# study.optimize(run, n_trials=400)

In [ ]:
# study.best_params

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
useful_features = ["pred_1", "pred_2"]
df_test = df_test[useful_features]
final_test_predictions = []
final_valid_predictions = {}
# final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    params = {
        'learning_rate': 0.08837982506397186,
        'reg_lambda': 1.5986606673969327e-06,
        'reg_alpha': 4.844826299110059e-08,
        'subsample': 0.4493238068204164,
        'colsample_bytree': 0.9987937142187765,
        'max_depth': 2,
        'random_state':fold,
        'num_parallel_tree':200,
        'tree_method':'gpu_hist',
        'gpu_id':0,
        'predictor':'gpu_predictor'
    }
    
    model = XGBRegressor(**params)
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)

    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_3"]
final_valid_predictions.to_csv("train_pred_3.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_3"]
sample_submission.to_csv("test_pred_3.csv", index=False)

# Stack 2

In [ ]:
df = pd.read_csv("../input/train-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

df1 = pd.read_csv("../input/starter-stack/train_pred_1.csv")
df2 = pd.read_csv("../input/starter-stack/train_pred_2.csv")
df3 = pd.read_csv("train_pred_3.csv")

df_test1 = pd.read_csv("../input/starter-stack/test_pred_1.csv")
df_test2 = pd.read_csv("../input/starter-stack/test_pred_2.csv")
df_test3 = pd.read_csv("test_pred_3.csv")

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")

df.head()

In [ ]:
useful_features = ["pred_1", "pred_2","pred_3"]
df_test = df_test[useful_features]

# study = optuna.create_study(direction="minimize")
# study.optimize(run, n_trials=400)

In [ ]:
# study.best_params

In [ ]:
useful_features = ["pred_1", "pred_2","pred_3"]
df_test = df_test[useful_features]
final_test_predictions = []
final_valid_predictions = {}
# final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = Ridge(alpha=4)
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)


    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_4"]
final_valid_predictions.to_csv("train_pred_4.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_4"]
sample_submission.to_csv("test_pred_4.csv", index=False)

# Stack 3

In [ ]:
df = pd.read_csv("../input/train-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

df1 = pd.read_csv("../input/starter-stack/train_pred_1.csv")
df2 = pd.read_csv("../input/starter-stack/train_pred_2.csv")
df3 = pd.read_csv("train_pred_3.csv")
df4 = pd.read_csv("train_pred_4.csv")

df_test1 = pd.read_csv("../input/starter-stack/test_pred_1.csv")
df_test2 = pd.read_csv("../input/starter-stack/test_pred_2.csv")
df_test3 = pd.read_csv("test_pred_3.csv")
df_test4 = pd.read_csv("test_pred_4.csv")

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")
df = df.merge(df4, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")
df_test = df_test.merge(df_test4, on="id", how="left")

df.head()

In [ ]:
useful_features = ['pred_1','pred_2','pred_3','pred_4']
df_test = df_test[useful_features]
final_test_predictions = []
final_valid_predictions = {}
# final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = Ridge(0.0001)
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)


    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_5"]
final_valid_predictions.to_csv("train_pred_5.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_5"]
sample_submission.to_csv("test_pred_5.csv", index=False)

# Stack 4

In [ ]:
df = pd.read_csv("../input/train-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

df1 = pd.read_csv("../input/starter-stack/train_pred_1.csv")
df2 = pd.read_csv("../input/starter-stack/train_pred_2.csv")
df3 = pd.read_csv("train_pred_3.csv")
df4 = pd.read_csv("train_pred_4.csv")
df5 = pd.read_csv("train_pred_5.csv")

df_test1 = pd.read_csv("../input/starter-stack/test_pred_1.csv")
df_test2 = pd.read_csv("../input/starter-stack/test_pred_2.csv")
df_test3 = pd.read_csv("test_pred_3.csv")
df_test4 = pd.read_csv("test_pred_4.csv")
df_test5 = pd.read_csv("test_pred_5.csv")

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")
df = df.merge(df4, on="id", how="left")
df = df.merge(df5, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")
df_test = df_test.merge(df_test4, on="id", how="left")
df_test = df_test.merge(df_test5, on="id", how="left")

df_test.head()

In [ ]:
useful_features = ["pred_1", "pred_2","pred_3","pred_4","pred_5"]
df_test = df_test[useful_features]
final_test_predictions = []
final_valid_predictions = {}
final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = Ridge(0.0001)
    model.fit(xtrain, ytrain)
    
    model.fit(xtrain, ytrain)
    
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)


    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)


In [ ]:
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")
sample_submission.head()

In [ ]:
sample_submission.target = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.to_csv("sub14.csv", index=False)